# Segmentation of Metro Vancouver Shops based on their categories and area income

# Import Necessary libraries

In [1]:
import pandas as pd
import folium
import numpy as np
import requests

# Read the CSV contains Metro Vancouver FSAs

In [2]:
df = pd.read_csv('MetroVancouver_FSA_Code.csv')

# Query Geolocation Service to find FSA latitude and longitude

In [3]:
def find_fsa_lat_lon(row):
    url = 'http://geogratis.gc.ca/services/geolocation/en/locate?q={}'.format(row['Code'])
    coordinates = requests.get(url).json()[0]['geometry']['coordinates']
    row['lat']= coordinates[1]
    row['lon']= coordinates[0]
    return row

df = df.apply(find_fsa_lat_lon, axis=1)
df.rename(columns={'Code':'FSA'}, inplace=True)

In [4]:
# Extract Coordinates of each FSA by using their mean coordinates
fsa_latlon = df[['FSA','lat','lon']]

# Read the Census 2016 Profile and extract the median income

In [5]:
fsa_income = pd.read_csv('census_2016_by_fsa/98-401-X2016046_English_CSV_data.csv').rename(columns={
                    'Member ID: Profile of Forward Sortation Areas (2247)':'DIM_ID',
                    'Dim: Sex (3): Member ID: [1]: Total - Sex':'Value',
                    'DIM: Profile of Forward Sortation Areas (2247)':'DIM'})
fsa_income= fsa_income[fsa_income['DIM_ID']==801]
fsa_income.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,DIM,DIM_ID,Notes: Profile of Forward Sortation Areas (2247),Value,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
800,2016,01,0,Canada,4.0,5.1,20000,01,Median total income of economic families in 20...,801,NaN,88306,...,...
3047,2016,A0A,2,A0A,4.2,9.1,0,01A0A,Median total income of economic families in 20...,801,NaN,80074,...,...
5294,2016,A0B,2,A0B,4.1,10.1,10,01A0B,Median total income of economic families in 20...,801,NaN,72917,...,...
7541,2016,A0C,2,A0C,4.1,8.6,0,01A0C,Median total income of economic families in 20...,801,NaN,62969,...,...
9788,2016,A0E,2,A0E,4.2,8.1,0,01A0E,Median total income of economic families in 20...,801,NaN,77716,...,...


# Merge Income with FSA

In [6]:
fsa_facts = fsa_latlon.merge(fsa_income,left_on='FSA', right_on='GEO_NAME', how='left')

In [7]:
fsa_facts=fsa_facts[['FSA','lat','lon','DIM', 'DIM_ID', 'Value']]

In [8]:
# Drop those without income information
fsa_facts['Value']=pd.to_numeric(fsa_facts['Value'], errors='coerce')
fsa_facts.dropna(inplace=True)

In [9]:
fsa_facts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 84
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   FSA     82 non-null     object 
 1   lat     82 non-null     float64
 2   lon     82 non-null     float64
 3   DIM     82 non-null     object 
 4   DIM_ID  82 non-null     int64  
 5   Value   82 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 4.5+ KB


# Normalize income

In [10]:
from sklearn import preprocessing

x = fsa_facts['Value'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
fsa_facts['value_norm'] = x_scaled

# Define the function to retreive venues information from Foursquare API

In [11]:
# Re-use the functions in the Lab. Add query=restaurant to limit results to restaurants
def getNearbyVenues(fsas, latitudes, longitudes, radius=5000):
    import requests
    
    CLIENT_ID = 'GI1CV0OGIPJVB2SOW4TL03H1OBYE0KH5BSJTWHW2SFXISANF' # your Foursquare ID
    CLIENT_SECRET = 'BENPMPAJAD5M3RD0O1N4LHLIQTXPBOJBFXU3M0MSJZXM0YOW' # your Foursquare Secret
    VERSION = '20201101' # Foursquare API version
    LIMIT = 50 # A default Foursquare API limit value
    venues_list=[]
    for fsa, lat, lng in zip(fsas, latitudes, longitudes):
        print(fsa)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=shops'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            fsa, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['shortName']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['FSA', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
# type your answer here
van_venues = getNearbyVenues(fsas=fsa_facts['FSA'],
                                   latitudes=fsa_facts['lat'],
                                   longitudes=fsa_facts['lon']
                                  )

V2P
V2R
V2S
V2T
V3C
V3E
V3J
V3K
V3L
V3M
V3R
V3S
V3T
V3V
V3W
V3X
V4C
V4E
V4K
V4L
V4M
V4N
V4P
V5C
V5E
V5G
V5H
V5J
V5K
V5L
V5M
V5N
V5P
V5R
V5S
V5T
V5V
V5W
V5X
V5Y
V5Z
V6C
V6E
V6G
V6H
V6J
V6K
V6L
V6M
V6N
V6P
V6R
V6S
V6V
V6W
V6X
V6Y
V6Z
V7B
V7C
V7E
V7G
V7H
V7J
V7K
V7L
V7M
V7N
V7P
V7R
V7S
V7T
V7V
V7W
V4A
V6T
V3B
V5A
V5B
V6A
V6B
V7A


In [13]:
# one hot encoding
van_onehot = pd.get_dummies(van_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
van_onehot['FSA'] = van_venues['FSA'] 

# move neighborhood column to the first column
fixed_columns = [van_onehot.columns[-1]] + list(van_onehot.columns[:-1])
van_onehot = van_onehot[fixed_columns]

van_onehot.head()

,FSA,ATM,Accessories,Apparel,Arts & Crafts,Auto Garage,Auto Workshop,Baby Store,Beer Store,Big Box Store,...,Sporting Goods,Supermarket,Supplement Shop,Thrift / Vintage,Toys & Games,Video Games,Video Store,Warehouse Store,Wine Shop,Women's Store
0,V2P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,V2P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,V2P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,V2P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,V2P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
van_grouped = van_onehot.groupby('FSA').mean().reset_index()
van_grouped.head()

,FSA,ATM,Accessories,Apparel,Arts & Crafts,Auto Garage,Auto Workshop,Baby Store,Beer Store,Big Box Store,...,Sporting Goods,Supermarket,Supplement Shop,Thrift / Vintage,Toys & Games,Video Games,Video Store,Warehouse Store,Wine Shop,Women's Store
0,V2P,0.00,0.00,0.00,0.0,0.028571,0.057143,0.0,0.0,0.00,...,0.028571,0.00,0.0,0.028571,0.0,0.00,0.0,0.00,0.0,0.0
1,V2R,0.00,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.00,...,0.000000,0.00,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.0
2,V2S,0.00,0.04,0.06,0.0,0.000000,0.040000,0.0,0.0,0.00,...,0.040000,0.02,0.0,0.020000,0.0,0.02,0.0,0.02,0.0,0.0
3,V2T,0.02,0.04,0.08,0.0,0.020000,0.000000,0.0,0.0,0.02,...,0.020000,0.02,0.0,0.020000,0.0,0.00,0.0,0.00,0.0,0.0
4,V3B,0.00,0.00,0.02,0.0,0.000000,0.000000,0.0,0.0,0.02,...,0.020000,0.06,0.0,0.000000,0.0,0.02,0.0,0.02,0.0,0.0


## Define a method to sort the venues in descending order.

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['FSA']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
van_grouped_sorted = pd.DataFrame(columns=columns)
van_grouped_sorted['FSA'] = van_grouped['FSA']

for ind in np.arange(van_grouped.shape[0]):
    van_grouped_sorted.iloc[ind, 1:] = return_most_common_venues(van_grouped.iloc[ind, :], num_top_venues)

van_grouped_sorted.head()

,FSA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V2P,Pharmacy,Grocery Store,Liquor Store,Auto Workshop,Medical Supply,Health & Beauty,Convenience Store,Pet Store,Insurance Office,Kids Store
1,V2R,Home Services,IT Services,Lawyer,Pet Store,Convenience Store,Flower Shop,Electronics,Fabric Shop,Farmer's Market,Fish Market
2,V2S,Pharmacy,Grocery Store,Furniture / Home,Mobile Phones,Electronics,Mall,Apparel,Sporting Goods,Auto Workshop,Department Store
3,V2T,Mobile Phones,Pharmacy,Apparel,Electronics,Insurance Office,Mall,Grocery Store,Print Shop,Business Services,Department Store
4,V3B,Grocery Store,Pharmacy,Convenience Store,Supermarket,Mall,Liquor Store,Farmer's Market,Furniture / Home,Electronics,Mobile Phones


## Run _k_-means to cluster the neighborhood into 5 clusters.

In [17]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 8

van_grouped_clustering = van_grouped.drop('FSA', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(van_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 5, 5, 1, 1, 5, 1, 7, 1], dtype=int32)

In [18]:
# add clustering labels
van_grouped_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
van_grouped_sorted.head()

,Cluster Labels,FSA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,V2P,Pharmacy,Grocery Store,Liquor Store,Auto Workshop,Medical Supply,Health & Beauty,Convenience Store,Pet Store,Insurance Office,Kids Store
1,4,V2R,Home Services,IT Services,Lawyer,Pet Store,Convenience Store,Flower Shop,Electronics,Fabric Shop,Farmer's Market,Fish Market
2,5,V2S,Pharmacy,Grocery Store,Furniture / Home,Mobile Phones,Electronics,Mall,Apparel,Sporting Goods,Auto Workshop,Department Store
3,5,V2T,Mobile Phones,Pharmacy,Apparel,Electronics,Insurance Office,Mall,Grocery Store,Print Shop,Business Services,Department Store
4,1,V3B,Grocery Store,Pharmacy,Convenience Store,Supermarket,Mall,Liquor Store,Farmer's Market,Furniture / Home,Electronics,Mobile Phones


In [19]:
# merge van_grouped_sorted with fsa_latlon to add latitude/longitude for each FSA
van_merged = fsa_facts.merge(van_grouped_sorted, left_on='FSA', right_on='FSA', how='inner')
van_merged.head()

,FSA,lat,lon,DIM,DIM_ID,Value,value_norm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V2P,49.191250,-121.907188,Median total income of economic families in 20...,801,70100.0,0.089482,1,Pharmacy,Grocery Store,Liquor Store,Auto Workshop,Medical Supply,Health & Beauty,Convenience Store,Pet Store,Insurance Office,Kids Store
1,V2R,49.056461,-121.971657,Median total income of economic families in 20...,801,87687.0,0.274951,4,Home Services,IT Services,Lawyer,Pet Store,Convenience Store,Flower Shop,Electronics,Fabric Shop,Farmer's Market,Fish Market
2,V2S,49.047405,-122.286743,Median total income of economic families in 20...,801,82516.0,0.220419,5,Pharmacy,Grocery Store,Furniture / Home,Mobile Phones,Electronics,Mall,Apparel,Sporting Goods,Auto Workshop,Department Store
3,V2T,49.043354,-122.348457,Median total income of economic families in 20...,801,78680.0,0.179965,5,Mobile Phones,Pharmacy,Apparel,Electronics,Insurance Office,Mall,Grocery Store,Print Shop,Business Services,Department Store
4,V3C,49.256513,-122.782005,Median total income of economic families in 20...,801,100547.0,0.410571,1,Pharmacy,Grocery Store,Convenience Store,Supermarket,Department Store,Mobile Phones,Discount Store,Pet Store,Mall,Farmer's Market


# Draw the map

In [24]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

# create map
map_clusters = folium.Map(location=[van_merged['lat'].mean(), van_merged['lon'].mean()], 
                          zoom_start=12,
                          tiles='Stamen Toner')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, median_income_norm in zip(van_merged['lat'], van_merged['lon'], van_merged['FSA'], van_merged['Cluster Labels'], van_merged['value_norm']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=median_income_norm*30,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
c1 = van_merged.loc[van_merged['Cluster Labels'] == 0, van_merged.columns[[1] + list(range(5, van_merged.shape[1]))]]
c1.head(10)
print(c1.Value.mean())

95407.73684210527


In [28]:
c2 = van_merged.loc[van_merged['Cluster Labels'] == 5, van_merged.columns[[1] + list(range(5, van_merged.shape[1]))]]
c2.head(10)
# print(c2.Value.mean())

,lat,Value,value_norm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,49.047405,82516.0,0.220419,5,Pharmacy,Grocery Store,Furniture / Home,Mobile Phones,Electronics,Mall,Apparel,Sporting Goods,Auto Workshop,Department Store
3,49.043354,78680.0,0.179965,5,Mobile Phones,Pharmacy,Apparel,Electronics,Insurance Office,Mall,Grocery Store,Print Shop,Business Services,Department Store
5,49.303803,99467.0,0.399182,5,Grocery Store,Apparel,Pharmacy,Convenience Store,Pet Store,Electronics,Sporting Goods,Farmer's Market,Discount Store,Mall
10,49.214794,77105.0,0.163355,5,Apparel,Grocery Store,Big Box Store,Electronics,Department Store,Supermarket,Pharmacy,Furniture / Home,Discount Store,Convenience Store
12,49.186653,70172.0,0.090241,5,Grocery Store,Pharmacy,Discount Store,Apparel,Liquor Store,Supermarket,Mall,Shoes,Pet Store,Toys & Games
19,49.018356,111019.0,0.521007,5,Apparel,Grocery Store,Sporting Goods,Insurance Office,Pet Store,Convenience Store,Department Store,Men's Store,Medical Supply,Mall
20,49.031330,118960.0,0.604752,5,Apparel,Sporting Goods,Chocolate Shop,Women's Store,Toys & Games,Men's Store,Department Store,Grocery Store,Accessories,IT Services
22,49.069243,104923.0,0.456720,5,Grocery Store,Pharmacy,Mall,Apparel,Sporting Goods,Furniture / Home,Supermarket,Pet Store,Farmer's Market,Kids Store
24,49.227497,79839.0,0.192188,5,Grocery Store,Liquor Store,Supermarket,Chocolate Shop,Apparel,Pharmacy,Furniture / Home,Mall,Discount Store,Bookstore
25,49.250175,96784.0,0.370887,5,Liquor Store,Pharmacy,Grocery Store,Pet Store,Toys & Games,Supermarket,Furniture / Home,Chocolate Shop,Electronics,Bookstore
